# <center> Introduction to Spark In-memory Computing via Python PySpark </center>

In [3]:
import sys
import os
import pyspark

print(os.environ['SPARK_HOME'])
print(os.environ['SPARK_MASTER_HOST'])
print(os.environ['SPARK_MASTER_PORT'])
print(os.environ['SPARK_MASTER_WEBUI_PORT'])

/ifs/opt/spark/3.5.0
node043
51679
53282


### Airlines Data

In [7]:
sqlContext = SparkSession.builder.appName("airline SQL").getOrCreate()
sqlContext

In [4]:
!ls -lh $HOME/data/airlines/data/

total 6.5G
-rw-rw-r-- 1 lbn28 lbn28 544M Feb 10 13:37 2000.csv
-rw-rw-r-- 1 lbn28 lbn28 573M Feb 10 13:34 2001.csv
-rw-rw-r-- 1 lbn28 lbn28 506M Feb 10 13:39 2002.csv
-rw-rw-r-- 1 lbn28 lbn28 598M Feb 10 13:37 2003.csv
-rw-rw-r-- 1 lbn28 lbn28 639M Feb 10 13:38 2004.csv
-rw-rw-r-- 1 lbn28 lbn28 640M Feb 10 13:39 2005.csv
-rw-rw-r-- 1 lbn28 lbn28 641M Feb 10 13:35 2006.csv
-rw-rw-r-- 1 lbn28 lbn28 671M Feb 10 13:37 2007.csv
-rw-rw-r-- 1 lbn28 lbn28 658M Feb 10 13:36 2008.csv


In [10]:
airline_data = os.environ['HOME'] + "/data/airlines/data/"
airlines = sqlContext.read.format("com.databricks.spark.csv")\
    .option("header", "true")\
    .option("inferschema", "true")\
    .load(airline_data)\
    .cache()

In [11]:
%%time
airlines.count()

CPU times: user 0 ns, sys: 1.89 ms, total: 1.89 ms
Wall time: 7.93 s


59285457

In [12]:
%%time
airlines.count()

CPU times: user 0 ns, sys: 775 µs, total: 775 µs
Wall time: 170 ms


59285457

In [13]:
airlines.printSchema()

root
 |-- Year: integer (nullable = true)
 |-- Month: integer (nullable = true)
 |-- DayofMonth: integer (nullable = true)
 |-- DayOfWeek: integer (nullable = true)
 |-- DepTime: string (nullable = true)
 |-- CRSDepTime: integer (nullable = true)
 |-- ArrTime: string (nullable = true)
 |-- CRSArrTime: integer (nullable = true)
 |-- UniqueCarrier: string (nullable = true)
 |-- FlightNum: integer (nullable = true)
 |-- TailNum: string (nullable = true)
 |-- ActualElapsedTime: string (nullable = true)
 |-- CRSElapsedTime: string (nullable = true)
 |-- AirTime: string (nullable = true)
 |-- ArrDelay: string (nullable = true)
 |-- DepDelay: string (nullable = true)
 |-- Origin: string (nullable = true)
 |-- Dest: string (nullable = true)
 |-- Distance: integer (nullable = true)
 |-- TaxiIn: string (nullable = true)
 |-- TaxiOut: string (nullable = true)
 |-- Cancelled: integer (nullable = true)
 |-- CancellationCode: string (nullable = true)
 |-- Diverted: integer (nullable = true)
 |-- Car

You can interact with a DataFrame via SQLContext using SQL statements by registering the DataFrame as a table

In [15]:
#airlines.registerTempTable("airlines")
airlines.createOrReplaceTempView("airlines")

*How many unique airlines are there?*

In [25]:
uniqueAirline = sqlContext.sql("SELECT DISTINCT UniqueCarrier FROM airlines")
uniqueAirline.show(5)

+-------------+
|UniqueCarrier|
+-------------+
|           UA|
|           AA|
|           NW|
|           EV|
|           B6|
+-------------+
only showing top 5 rows



*Calculate how many flights completed by each carrier over time*

In [24]:
%%time
carrierFlightCount = sqlContext.sql("SELECT UniqueCarrier, COUNT(UniqueCarrier) AS FlightCount \
                                    FROM airlines GROUP BY UniqueCarrier")
carrierFlightCount.show(5)

+-------------+-----------+
|UniqueCarrier|FlightCount|
+-------------+-----------+
|           UA|    5094635|
|           AA|    6318386|
|           NW|    4280049|
|           EV|    1697172|
|           B6|     811341|
+-------------+-----------+
only showing top 5 rows

CPU times: user 610 µs, sys: 904 µs, total: 1.51 ms
Wall time: 271 ms


*How do you display full carrier names?*

In [19]:
carriers = sqlContext.read.format("com.databricks.spark.csv")\
    .option("header", "true").option("inferschema", "true")\
    .load(os.environ['HOME'] + "/data/airlines/metadata/carriers.csv").cache()
carriers.registerTempTable("carriers")

In [20]:
carriers.printSchema()

root
 |-- Code: string (nullable = true)
 |-- Description: string (nullable = true)



In [26]:
%%time
carrierFlightCountFullName = sqlContext.sql("SELECT c.Description, a.UniqueCarrier, COUNT(a.UniqueCarrier) AS FlightCount \
                                    FROM airlines AS a \
                                    INNER JOIN carriers AS c \
                                    ON c.Code = a.UniqueCarrier \
                                    GROUP BY a.UniqueCarrier, c.Description \
                                    ORDER BY a.UniqueCarrier")
carrierFlightCountFullName.show(5)

+--------------------+-------------+-----------+
|         Description|UniqueCarrier|FlightCount|
+--------------------+-------------+-----------+
|Pinnacle Airlines...|           9E|     521059|
|American Airlines...|           AA|    6318386|
| Aloha Airlines Inc.|           AQ|     154381|
|Alaska Airlines Inc.|           AS|    1427189|
|     JetBlue Airways|           B6|     811341|
+--------------------+-------------+-----------+
only showing top 5 rows

CPU times: user 1.41 ms, sys: 539 µs, total: 1.95 ms
Wall time: 2.71 s


*What is the averaged departure delay time for each airline?*

In [22]:
%%time
avgDepartureDelay = sqlContext.sql("SELECT FIRST(c.Description), FIRST(a.UniqueCarrier), AVG(a.DepDelay) AS AvgDepDelay \
                                    FROM airlines AS a \
                                    INNER JOIN carriers AS c \
                                    ON c.Code = a.UniqueCarrier \
                                    GROUP BY a.UniqueCarrier \
                                    ORDER BY a.UniqueCarrier")
avgDepartureDelay.show()

+--------------------+--------------------+-------------------+
|  first(Description)|first(UniqueCarrier)|        AvgDepDelay|
+--------------------+--------------------+-------------------+
|Pinnacle Airlines...|                  9E| 7.9279144892173035|
|American Airlines...|                  AA|   9.31183542798288|
| Aloha Airlines Inc.|                  AQ| 1.5993176899118409|
|Alaska Airlines Inc.|                  AS|   9.59814943329714|
|     JetBlue Airways|                  B6| 11.262714178314551|
|Continental Air L...|                  CO|  7.772605178542145|
|    Independence Air|                  DH|  9.612639389688926|
|Delta Air Lines Inc.|                  DL| 7.4335417135660515|
|Atlantic Southeas...|                  EV| 13.483736343326541|
|Frontier Airlines...|                  F9|  6.096932123645889|
|AirTran Airways C...|                  FL|  10.27801937883596|
|Hawaiian Airlines...|                  HA|-0.5165400834606493|
|America West Airl...|                  

In [ ]:
airlines.unpersist()